In [1]:
!pip install -q datasets sacrebleu rouge-score

from google.colab import drive
drive.mount("/content/drive", force_remount=True)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.9 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
import os
import json
import time
import math
import random
from collections import Counter

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset
import sacrebleu
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)


Device: cuda


In [3]:
BASE_DIR = "/content/drive/MyDrive/TakeHome543/Question3"

DIRS = {
    "data": os.path.join(BASE_DIR, "data"),
    "vocab": os.path.join(BASE_DIR, "vocab"),
    "checkpoints": os.path.join(BASE_DIR, "checkpoints"),
    "logs": os.path.join(BASE_DIR, "logs"),
    "figs": os.path.join(BASE_DIR, "figs"),
}

for p in DIRS.values():
    os.makedirs(p, exist_ok=True)

print("Base directory:", BASE_DIR)


Base directory: /content/drive/MyDrive/TakeHome543/Question3


In [5]:
MIN_FREQ = 3
MAX_VOCAB_SIZE = 20000
MAX_LEN = 30

SPECIAL_TOKENS = {"<pad>":0, "<bos>":1, "<eos>":2, "<unk>":3}
PAD_IDX = SPECIAL_TOKENS["<pad>"]
BOS_IDX = SPECIAL_TOKENS["<bos>"]
EOS_IDX = SPECIAL_TOKENS["<eos>"]
UNK_IDX = SPECIAL_TOKENS["<unk>"]

BATCH_SIZE = 64
EMB_SIZE = 256
HIDDEN_SIZE = 256
ENC_LAYERS = 1
DEC_LAYERS = 1
DROPOUT = 0.2
LR = 1e-3
MAX_DECODE_LEN = MAX_LEN

# Training epochs
BAHDANAU_EPOCHS = 10
TRANSFORMER_EPOCHS = 10


In [6]:
def simple_tokenize(text: str):
    return text.strip().lower().split()


In [7]:
def load_quora_paraphrase_splits(seed=42):
    raw = load_dataset("AlekseyKorshuk/quora-question-pairs")
    # only duplicate (= paraphrase) pairs
    dup = raw["train"].filter(lambda ex: ex["is_duplicate"] == 1)
    dup = dup.shuffle(seed=seed)

    n = len(dup)
    n_train = int(0.8 * n)
    n_valid = int(0.1 * n)

    train_split = dup.select(range(n_train))
    valid_split = dup.select(range(n_train, n_train+n_valid))
    test_split = dup.select(range(n_train+n_valid, n))

    return train_split, valid_split, test_split

train_ds, valid_ds, test_ds = load_quora_paraphrase_splits()
print("Dataset sizes:", len(train_ds), len(valid_ds), len(test_ds))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/41.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

Filter:   0%|          | 0/404290 [00:00<?, ? examples/s]

Dataset sizes: 119410 14926 14927


In [8]:
def build_vocab(train_split, min_freq, max_vocab_size):
    counter = Counter()
    for ex in train_split:
        counter.update(simple_tokenize(ex["question1"]))
        counter.update(simple_tokenize(ex["question2"]))

    filtered = [(w,c) for w,c in counter.items() if c >= min_freq]
    filtered.sort(key=lambda x: x[1], reverse=True)
    filtered = filtered[: (max_vocab_size - len(SPECIAL_TOKENS))]

    itos = [None]*len(SPECIAL_TOKENS) + [w for w,_ in filtered]
    stoi = {}

    # first: special tokens
    for tok, idx in SPECIAL_TOKENS.items():
        itos[idx] = tok
        stoi[tok] = idx

    # the rest
    for idx, tok in enumerate(itos):
        if tok not in stoi:
            stoi[tok] = idx

    return stoi, itos

stoi, itos = build_vocab(train_ds, MIN_FREQ, MAX_VOCAB_SIZE)
VOCAB_SIZE = len(itos)
VOCAB_SIZE


20000

In [9]:
vocab_path = os.path.join(DIRS["vocab"], "vocab.json")
with open(vocab_path, "w", encoding="utf-8") as f:
    json.dump({"stoi": stoi, "itos": itos}, f, indent=2)

print("Saved vocab to:", vocab_path)


Saved vocab to: /content/drive/MyDrive/TakeHome543/Question3/vocab/vocab.json


In [10]:
def encode(text):
    tokens = simple_tokenize(text)
    tokens = tokens[: MAX_LEN-2]

    ids = [BOS_IDX] + [stoi.get(t, UNK_IDX) for t in tokens] + [EOS_IDX]
    if len(ids) < MAX_LEN:
        ids += [PAD_IDX] * (MAX_LEN - len(ids))
    return ids


def decode(ids):
    out = []
    for i in ids:
        if i == PAD_IDX:
            continue
        if i == EOS_IDX:
            break
        out.append(itos[i] if i < len(itos) else "<unk>")
    return out


In [11]:
def save_jsonl(split, path, max_items=20000):
    with open(path, "w", encoding="utf-8") as f:
        for i, ex in enumerate(split):
            if i >= max_items:
                break
            obj = {
                "question1": ex["question1"],
                "question2": ex["question2"],
                "src_ids": encode(ex["question1"]),
                "tgt_ids": encode(ex["question2"]),
            }
            f.write(json.dumps(obj) + "\n")

save_jsonl(train_ds, os.path.join(DIRS["data"], "train.jsonl"))
save_jsonl(valid_ds, os.path.join(DIRS["data"], "valid.jsonl"))
save_jsonl(test_ds,  os.path.join(DIRS["data"], "test.jsonl"))

print("JSONL snapshots saved.")


JSONL snapshots saved.


In [12]:
class QuoraDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        ex = self.data[idx]
        src = encode(ex["question1"])
        tgt = encode(ex["question2"])

        return {
            "src": torch.tensor(src),
            "dec_in": torch.tensor(tgt[:-1]),
            "dec_out": torch.tensor(tgt[1:]),
            "src_text": ex["question1"],
            "tgt_text": ex["question2"],
        }

train_loader = DataLoader(QuoraDataset(train_ds), batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(QuoraDataset(valid_ds), batch_size=BATCH_SIZE)
test_loader  = DataLoader(QuoraDataset(test_ds),  batch_size=BATCH_SIZE)

print("Batches:", len(train_loader), len(valid_loader), len(test_loader))


Batches: 1866 234 234


In [13]:
class BahdanauAttention(nn.Module):
    """
    Additive attention (Bahdanau et al., 2015)
    """
    def __init__(self, hidden_size):
        super().__init__()
        self.W_h = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_s = nn.Linear(hidden_size, hidden_size, bias=False)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, decoder_hidden, encoder_outputs, mask=None):
        # decoder_hidden : [B, H]
        # encoder_outputs: [B, T, H]
        proj_enc = self.W_h(encoder_outputs)
        proj_dec = self.W_s(decoder_hidden).unsqueeze(1)
        scores = self.v(torch.tanh(proj_enc + proj_dec)).squeeze(-1)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn = torch.softmax(scores, dim=-1)
        context = torch.bmm(attn.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attn


In [14]:
class Encoder(nn.Module):
    """
    BiLSTM Encoder
    """
    def __init__(self, vocab_size, emb_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.lstm = nn.LSTM(
            emb_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True,
        )
        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src_ids):
        mask = (src_ids != PAD_IDX).int()
        emb = self.dropout(self.embedding(src_ids))
        outputs, (h_n, c_n) = self.lstm(emb)

        outputs = torch.tanh(self.fc(outputs))

        h_cat = torch.cat((h_n[-2], h_n[-1]), dim=-1)
        c_cat = torch.cat((c_n[-2], c_n[-1]), dim=-1)

        h0 = torch.tanh(self.fc(h_cat)).unsqueeze(0)
        c0 = torch.tanh(self.fc(c_cat)).unsqueeze(0)

        return outputs, (h0, c0), mask


In [15]:
class Decoder(nn.Module):
    """
    Decoder with Bahdanau Attention
    """
    def __init__(self, vocab_size, emb_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.lstm = nn.LSTM(
            emb_size + hidden_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.fc_out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.attn = BahdanauAttention(hidden_size)

    def forward(self, dec_in_ids, encoder_outputs, encoder_hidden, src_mask):
        emb = self.dropout(self.embedding(dec_in_ids))
        batch_size, tgt_len, _ = emb.size()

        h, c = encoder_hidden
        h = h.contiguous()
        c = c.contiguous()

        logits_list = []
        attn_list = []

        for t in range(tgt_len):
            emb_t = emb[:, t, :]

            context, attn_weights = self.attn(h[-1], encoder_outputs, src_mask)

            lstm_input = torch.cat([emb_t, context], dim=-1).unsqueeze(1)
            output, (h, c) = self.lstm(lstm_input, (h, c))
            logit_t = self.fc_out(output.squeeze(1))

            logits_list.append(logit_t)
            attn_list.append(attn_weights)

        logits = torch.stack(logits_list, dim=1)      # [B, T, V]
        attentions = torch.stack(attn_list, dim=1)    # [B, T, src_len]

        return logits, attentions


In [16]:
class Seq2Seq(nn.Module):
    """
    Full Encoder-Decoder with Bahdanau Attention
    """
    def __init__(self):
        super().__init__()
        self.encoder = Encoder(
            vocab_size=VOCAB_SIZE,
            emb_size=EMB_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=ENC_LAYERS,
            dropout=DROPOUT,
        )
        self.decoder = Decoder(
            vocab_size=VOCAB_SIZE,
            emb_size=EMB_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=DEC_LAYERS,
            dropout=DROPOUT,
        )

    def forward(self, src, dec_in):
        enc_out, enc_hidden, mask = self.encoder(src)
        return self.decoder(dec_in, enc_out, enc_hidden, mask)


In [17]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0

    for batch in loader:
        src = batch["src"].to(DEVICE)
        dec_in = batch["dec_in"].to(DEVICE)
        dec_out = batch["dec_out"].to(DEVICE)

        optimizer.zero_grad()
        logits, _ = model(src, dec_in)

        B,T,V = logits.shape
        loss = criterion(logits.reshape(B*T, V), dec_out.reshape(B*T))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)


def eval_loss(model, loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            src = batch["src"].to(DEVICE)
            dec_in = batch["dec_in"].to(DEVICE)
            dec_out = batch["dec_out"].to(DEVICE)

            logits, _ = model(src, dec_in)
            B,T,V = logits.shape
            loss = criterion(logits.reshape(B*T, V), dec_out.reshape(B*T))

            total_loss += loss.item()
    return total_loss / len(loader)


In [18]:
bahdanau_model = Seq2Seq().to(DEVICE)
optimizer = torch.optim.Adam(bahdanau_model.parameters(), lr=LR)

bahdanau_history = []

for epoch in range(1, BAHDANAU_EPOCHS + 1):
    t0 = time.time()
    train_l = train_one_epoch(bahdanau_model, train_loader, optimizer)
    valid_l = eval_loss(bahdanau_model, valid_loader)
    t1 = time.time()

    print(f"[Bahdanau] Epoch {epoch}/{BAHDANAU_EPOCHS} "
          f"| Train={train_l:.3f} | Valid={valid_l:.3f} | Time={t1-t0:.1f}s")

    bahdanau_history.append({
        "epoch": epoch,
        "train_loss": train_l,
        "valid_loss": valid_l,
        "time_sec": t1 - t0,
    })

torch.save(bahdanau_model.state_dict(), os.path.join(DIRS["checkpoints"], "bahdanau.pt"))
print("Bahdanau model saved.")


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[Bahdanau] Epoch 1/10 | Train=3.995 | Valid=3.016 | Time=180.2s
[Bahdanau] Epoch 2/10 | Train=2.745 | Valid=2.502 | Time=176.5s
[Bahdanau] Epoch 3/10 | Train=2.273 | Valid=2.273 | Time=177.0s
[Bahdanau] Epoch 4/10 | Train=1.991 | Valid=2.151 | Time=175.4s
[Bahdanau] Epoch 5/10 | Train=1.798 | Valid=2.077 | Time=175.9s
[Bahdanau] Epoch 6/10 | Train=1.655 | Valid=2.030 | Time=175.8s
[Bahdanau] Epoch 7/10 | Train=1.545 | Valid=1.999 | Time=173.1s
[Bahdanau] Epoch 8/10 | Train=1.456 | Valid=1.984 | Time=173.9s
[Bahdanau] Epoch 9/10 | Train=1.383 | Valid=1.973 | Time=176.2s
[Bahdanau] Epoch 10/10 | Train=1.321 | Valid=1.962 | Time=174.6s
Bahdanau model saved.


In [19]:
# 4) Transformer hyperparameters

D_MODEL = EMB_SIZE
TRANS_FF_SIZE = 512
TRANS_N_LAYERS_BASE = 3
TRANS_N_HEADS_BASE = 4
TRANS_DROPOUT = 0.1


In [20]:
class PositionalEncoding(nn.Module):
    """
    Standard sinusoidal positional encoding (Vaswani et al., 2017).
    """

    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2, dtype=torch.float32) *
            (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer("pe", pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [B, T, d_model]
        """
        T = x.size(1)
        return x + self.pe[:, :T, :]


In [21]:
class TransformerSeq2Seq(nn.Module):
    """
    Transformer-based encoder-decoder model for seq2seq paraphrase generation.

    - Token embeddings are shared between encoder and decoder.
    - PositionalEncoding adds order information.
    """

    def __init__(
        self,
        vocab_size: int,
        d_model: int = D_MODEL,
        nhead: int = TRANS_N_HEADS_BASE,
        num_encoder_layers: int = TRANS_N_LAYERS_BASE,
        num_decoder_layers: int = TRANS_N_LAYERS_BASE,
        dim_feedforward: int = TRANS_FF_SIZE,
        dropout: float = TRANS_DROPOUT,
    ):
        super().__init__()

        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=PAD_IDX)
        self.pos_encoder = PositionalEncoding(d_model, max_len=MAX_LEN)

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,  # work with [B, T, D]
        )

        self.generator = nn.Linear(d_model, vocab_size)

    def make_pad_mask(self, ids: torch.Tensor) -> torch.Tensor:
        """
        Create a key_padding_mask for padding tokens.

        ids: [B, T]
        returns: [B, T] bool mask with True at PAD positions.
        """
        return (ids == PAD_IDX)

    def make_causal_mask(self, size: int, device: torch.device) -> torch.Tensor:
        """
        Create a causal (subsequent) mask for the target sequence.
        Shape: [T, T], with -inf above diagonal.
        """
        mask = torch.triu(torch.ones(size, size, device=device), diagonal=1).bool()
        # nn.Transformer expects float mask with -inf where masked
        mask = mask.float().masked_fill(mask, float("-inf"))
        return mask

    def forward(self, src_ids: torch.Tensor, tgt_ids_in: torch.Tensor):
        """
        src_ids: [B, S]
        tgt_ids_in: [B, T]  (decoder input, shifted right)
        """
        B, S = src_ids.shape
        _, T = tgt_ids_in.shape

        src_pad_mask = self.make_pad_mask(src_ids)      # [B, S]
        tgt_pad_mask = self.make_pad_mask(tgt_ids_in)   # [B, T]
        tgt_mask = self.make_causal_mask(T, src_ids.device)  # [T, T]

        # token embeddings + positional encoding
        src_emb = self.embedding(src_ids) * math.sqrt(self.d_model)
        tgt_emb = self.embedding(tgt_ids_in) * math.sqrt(self.d_model)

        src_emb = self.pos_encoder(src_emb)
        tgt_emb = self.pos_encoder(tgt_emb)

        # Transformer forward
        out = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_pad_mask,
            tgt_key_padding_mask=tgt_pad_mask,
            memory_key_padding_mask=src_pad_mask,
        )  # [B, T, d_model]

        logits = self.generator(out)  # [B, T, vocab]
        return logits


In [22]:
def train_one_epoch_transformer(model, loader, optimizer):
    model.train()
    total_loss = 0.0

    for batch in loader:
        src = batch["src"].to(DEVICE)
        dec_in = batch["dec_in"].to(DEVICE)
        dec_out = batch["dec_out"].to(DEVICE)

        optimizer.zero_grad()
        logits = model(src, dec_in)  # [B, T, V]

        B, T, V = logits.shape
        loss = criterion(logits.reshape(B*T, V), dec_out.reshape(B*T))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


def eval_loss_transformer(model, loader):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for batch in loader:
            src = batch["src"].to(DEVICE)
            dec_in = batch["dec_in"].to(DEVICE)
            dec_out = batch["dec_out"].to(DEVICE)

            logits = model(src, dec_in)
            B, T, V = logits.shape
            loss = criterion(logits.reshape(B*T, V), dec_out.reshape(B*T))
            total_loss += loss.item()

    return total_loss / len(loader)


In [23]:
def greedy_generate_transformer(model, src_ids_batch, max_len: int = MAX_LEN):
    """
    Greedy decoding for Transformer model.
    src_ids_batch: [B, S]
    """
    model.eval()
    src = src_ids_batch.to(DEVICE)
    B = src.size(0)

    # decoder input starts with <bos>
    dec_in = torch.full((B, 1), BOS_IDX, dtype=torch.long, device=DEVICE)
    generated = [[] for _ in range(B)]

    with torch.no_grad():
        for _ in range(max_len - 1):
            logits = model(src, dec_in)
            next_ids = logits[:, -1, :].argmax(dim=-1)

            for i in range(B):
                generated[i].append(next_ids[i].item())

            dec_in = torch.cat([dec_in, next_ids.unsqueeze(1)], dim=1)

    return generated


In [24]:
transformer_model = TransformerSeq2Seq(
    vocab_size=VOCAB_SIZE,
    d_model=D_MODEL,
    nhead=TRANS_N_HEADS_BASE,
    num_encoder_layers=TRANS_N_LAYERS_BASE,
    num_decoder_layers=TRANS_N_LAYERS_BASE,
    dim_feedforward=TRANS_FF_SIZE,
    dropout=TRANS_DROPOUT,
).to(DEVICE)

optimizer_trans = torch.optim.Adam(transformer_model.parameters(), lr=LR)

transformer_history = []

for epoch in range(1, TRANSFORMER_EPOCHS + 1):
    t0 = time.time()
    train_l = train_one_epoch_transformer(transformer_model, train_loader, optimizer_trans)
    valid_l = eval_loss_transformer(transformer_model, valid_loader)
    t1 = time.time()

    print(f"[Transformer] Epoch {epoch}/{TRANSFORMER_EPOCHS} "
          f"| Train={train_l:.3f} | Valid={valid_l:.3f} | Time={t1-t0:.1f}s")

    transformer_history.append({
        "epoch": epoch,
        "train_loss": train_l,
        "valid_loss": valid_l,
        "time_sec": t1 - t0,
    })

torch.save(
    transformer_model.state_dict(),
    os.path.join(DIRS["checkpoints"], "transformer_base.pt")
)
print("Transformer base model saved.")


/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:515: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


[Transformer] Epoch 1/10 | Train=3.805 | Valid=2.890 | Time=120.4s
[Transformer] Epoch 2/10 | Train=2.691 | Valid=2.455 | Time=117.5s
[Transformer] Epoch 3/10 | Train=2.263 | Valid=2.240 | Time=117.2s
[Transformer] Epoch 4/10 | Train=2.004 | Valid=2.120 | Time=118.0s
[Transformer] Epoch 5/10 | Train=1.825 | Valid=2.059 | Time=117.3s
[Transformer] Epoch 6/10 | Train=1.700 | Valid=1.998 | Time=117.5s
[Transformer] Epoch 7/10 | Train=1.604 | Valid=1.998 | Time=118.2s
[Transformer] Epoch 8/10 | Train=1.531 | Valid=1.955 | Time=118.4s
[Transformer] Epoch 9/10 | Train=1.467 | Valid=1.951 | Time=119.9s
[Transformer] Epoch 10/10 | Train=1.411 | Valid=1.937 | Time=119.1s
Transformer base model saved.


In [25]:
def greedy_generate_bahdanau(model, src_ids_batch, max_len: int = MAX_LEN):
    """
    Greedy decoding for Bahdanau seq2seq model.
    src_ids_batch: [B, S]
    """
    model.eval()
    src = src_ids_batch.to(DEVICE)
    B = src.size(0)

    dec_in = torch.full((B, 1), BOS_IDX, dtype=torch.long, device=DEVICE)
    generated = [[] for _ in range(B)]

    with torch.no_grad():
        for _ in range(max_len - 1):
            logits, _ = model(src, dec_in)  # [B, T, V]
            next_ids = logits[:, -1, :].argmax(dim=-1)  # [B]

            for i in range(B):
                generated[i].append(next_ids[i].item())

            dec_in = torch.cat([dec_in, next_ids.unsqueeze(1)], dim=1)

    return generated


In [26]:
def loss_to_ppl(loss: float) -> float:
    return math.exp(loss)


In [27]:
def evaluate_model_text(
    model,
    loader,
    generate_fn,
    max_batches: int = 50,
    name: str = "model"
):
    """
    Compute BLEU, ROUGE-L, PPL, eval_time, max_gpu_mem for a given model.
    """
    model.eval()
    total_loss = 0.0
    batches = 0
    all_refs = []
    all_hyps = []

    # GPU memory reset (if CUDA is available)
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats(DEVICE)

    t0 = time.time()
    with torch.no_grad():
        for batch in loader:
            src = batch["src"].to(DEVICE)
            dec_in = batch["dec_in"].to(DEVICE)
            dec_out = batch["dec_out"].to(DEVICE)

            # forward for loss
            if isinstance(model, TransformerSeq2Seq):
                logits = model(src, dec_in)
            else:
                logits, _ = model(src, dec_in)

            B, T, V = logits.shape
            loss = criterion(logits.view(B*T, V), dec_out.view(B*T))
            total_loss += loss.item()
            batches += 1

            # generation
            gen_ids = generate_fn(model, src, max_len=MAX_LEN)

            for i in range(B):
                ref_tokens = decode(dec_out[i].cpu().tolist())
                hyp_tokens = decode(gen_ids[i])

                if len(ref_tokens) == 0 or len(hyp_tokens) == 0:
                    continue

                all_refs.append([" ".join(ref_tokens)])
                all_hyps.append(" ".join(hyp_tokens))

            if batches >= max_batches:
                break
    t1 = time.time()
    avg_loss = total_loss / max(1, batches)
    ppl = loss_to_ppl(avg_loss)

    if len(all_hyps) == 0:
        bleu_score = 0.0
        rouge_l = 0.0
    else:
        bleu = sacrebleu.corpus_bleu(all_hyps, list(zip(*all_refs)))
        bleu_score = float(bleu.score)

        scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
        rouge_scores = [
            scorer.score(r[0], h)["rougeL"].fmeasure
            for h, r in zip(all_hyps, all_refs)
        ]
        rouge_l = float(np.mean(rouge_scores))

    eval_time = t1 - t0
    if torch.cuda.is_available():
        max_mem = torch.cuda.max_memory_allocated(DEVICE) / (1024**2)
    else:
        max_mem = 0.0

    metrics = {
        "name": name,
        "bleu": bleu_score,
        "rougeL": rouge_l,
        "ppl": float(ppl),
        "eval_time_sec": float(eval_time),
        "max_gpu_mem_mb": float(max_mem),
        "avg_loss": float(avg_loss),
        "num_batches": int(batches),
    }
    return metrics


In [28]:
# Evaluate Bahdanau
bahdanau_metrics = evaluate_model_text(
    bahdanau_model,
    test_loader,
    generate_fn=greedy_generate_bahdanau,
    max_batches=50,
    name="bahdanau"
)
print("Bahdanau metrics:", bahdanau_metrics)

# Evaluate Transformer
transformer_metrics = evaluate_model_text(
    transformer_model,
    test_loader,
    generate_fn=greedy_generate_transformer,
    max_batches=50,
    name="transformer_base"
)
print("Transformer metrics:", transformer_metrics)

# Save histories and metrics
with open(os.path.join(DIRS["logs"], "bahdanau_history.json"), "w") as f:
    json.dump(bahdanau_history, f, indent=2)

with open(os.path.join(DIRS["logs"], "transformer_history.json"), "w") as f:
    json.dump(transformer_history, f, indent=2)

with open(os.path.join(DIRS["logs"], "bahdanau_metrics.json"), "w") as f:
    json.dump(bahdanau_metrics, f, indent=2)

with open(os.path.join(DIRS["logs"], "transformer_base_metrics.json"), "w") as f:
    json.dump(transformer_metrics, f, indent=2)

print("Saved metrics and histories into:", DIRS["logs"])


Bahdanau metrics: {'name': 'bahdanau', 'bleu': 26.733609022124973, 'rougeL': 0.5393148016451234, 'ppl': 7.295752522981305, 'eval_time_sec': 22.645774841308594, 'max_gpu_mem_mb': 1077.01513671875, 'avg_loss': 1.9872923326492309, 'num_batches': 50}
Transformer metrics: {'name': 'transformer_base', 'bleu': 24.61543636879578, 'rougeL': 0.5276724822415376, 'ppl': 7.090618626766887, 'eval_time_sec': 17.099522590637207, 'max_gpu_mem_mb': 936.2763671875, 'avg_loss': 1.9587725901603699, 'num_batches': 50}
Saved metrics and histories into: /content/drive/MyDrive/TakeHome543/Question3/logs


In [29]:
def plot_loss_curves(history, title, fig_name):
    epochs = [h["epoch"] for h in history]
    train_losses = [h["train_loss"] for h in history]
    valid_losses = [h["valid_loss"] for h in history]

    plt.figure()
    plt.plot(epochs, train_losses, label="Train loss")
    plt.plot(epochs, valid_losses, label="Valid loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(title)
    plt.legend()
    plt.grid(True)

    fig_path = os.path.join(DIRS["figs"], fig_name)
    plt.savefig(fig_path, dpi=200, bbox_inches="tight")
    plt.close()
    print("Saved figure:", fig_path)

plot_loss_curves(bahdanau_history, "Bahdanau Seq2Seq Loss", "bahdanau_loss.png")
plot_loss_curves(transformer_history, "Transformer Base Loss", "transformer_loss.png")


Saved figure: /content/drive/MyDrive/TakeHome543/Question3/figs/bahdanau_loss.png
Saved figure: /content/drive/MyDrive/TakeHome543/Question3/figs/transformer_loss.png


In [31]:
def show_sample_predictions(
    bahdanau_model,
    transformer_model,
    loader,
    num_examples: int = 5
):
    bahdanau_model.eval()
    transformer_model.eval()

    shown = 0
    with torch.no_grad():
        for batch in loader:
            src = batch["src"]
            dec_out = batch["dec_out"]
            src_texts = batch["src_text"]
            tgt_texts = batch["tgt_text"]

            B = src.size(0)

            # Bahdanau outputs
            bah_gen_ids = greedy_generate_bahdanau(bahdanau_model, src, max_len=MAX_LEN)
            # Transformer outputs
            trans_gen_ids = greedy_generate_transformer(transformer_model, src, max_len=MAX_LEN)

            for i in range(B):
                if shown >= num_examples:
                    return

                src_sent = src_texts[i]
                ref_tokens = decode(dec_out[i].cpu().tolist())
                ref_sent = " ".join(ref_tokens)
                bah_tokens = decode(bah_gen_ids[i])
                bah_sent = " ".join(bah_tokens)
                trans_tokens = decode(trans_gen_ids[i])
                trans_sent = " ".join(trans_tokens)

                print("="*80)
                print(f"Example {shown}")
                print("SRC:", src_sent)
                print("REF:", ref_sent)
                print("BAH:", bah_sent)
                print("TRA:", trans_sent)
                print("="*80)
                shown += 1

# show a few qualitative examples
show_sample_predictions(bahdanau_model, transformer_model, test_loader, num_examples=5)


Example 0
SRC: How can I loose weight naturally without exercise?
REF: can i lose weight without exercise?
BAH: how can i lose weight without doing exercise or diet?
TRA: how can i loose weight naturally without exercise?
Example 1
SRC: What are the best books to prepare for the OCPJP certification exam?
REF: which is the best book to study for the <unk> certification in java?
BAH: what are the best books for preparing for the <unk> entrance exam 2016?
TRA: what are the best books for learning certification for beginners?
Example 2
SRC: Are you single?
REF: are you single? why?
BAH: are you single?
TRA: are you single?
Example 3
SRC: What's the good name of a animation studio?
REF: what would be the best name for an animation studio that makes books into movies?
BAH: what is the best name of a pc game?
TRA: what is the best name of the animation software?
Example 4
SRC: How do I improve diet to gain weight? What are other options to gain weight?
REF: which foods help gain weight?
BAH: 

In [32]:

# 6) Ablation: Vary Layers & Heads

ABLATION_CONFIGS = [
    {"name": "L2_H2", "num_layers": 2, "nhead": 2},
    {"name": "L2_H4", "num_layers": 2, "nhead": 4},
    {"name": "L4_H4", "num_layers": 4, "nhead": 4},
    {"name": "L6_H8", "num_layers": 6, "nhead": 8},
]

ABLATION_EPOCHS = 5


def run_transformer_ablation(
    configs,
    epochs: int = ABLATION_EPOCHS,
    max_eval_batches: int = 30,
):
    """
    Run a small training loop for multiple Transformer variants
    and log BLEU / ROUGE-L / PPL / times / GPU memory.
    """
    results = []

    for cfg in configs:
        name = cfg["name"]
        n_layers = cfg["num_layers"]
        n_heads = cfg["nhead"]

        print("=" * 80)
        print(f"[Ablation] Config {name}: layers={n_layers}, heads={n_heads}")
        print("=" * 80)

        set_seed(42)

        model = TransformerSeq2Seq(
            vocab_size=VOCAB_SIZE,
            d_model=D_MODEL,
            nhead=n_heads,
            num_encoder_layers=n_layers,
            num_decoder_layers=n_layers,
            dim_feedforward=TRANS_FF_SIZE,
            dropout=TRANS_DROPOUT,
        ).to(DEVICE)

        optimizer = torch.optim.Adam(model.parameters(), lr=LR)

        history = []
        total_train_time = 0.0

        for epoch in range(1, epochs + 1):
            t0 = time.time()
            train_l = train_one_epoch_transformer(model, train_loader, optimizer)
            valid_l = eval_loss_transformer(model, valid_loader)
            t1 = time.time()

            total_train_time += (t1 - t0)

            print(
                f"[{name}] Epoch {epoch}/{epochs} "
                f"| Train={train_l:.3f} | Valid={valid_l:.3f} "
                f"| Time={t1-t0:.1f}s"
            )

            history.append(
                {
                    "epoch": epoch,
                    "train_loss": float(train_l),
                    "valid_loss": float(valid_l),
                    "time_sec": float(t1 - t0),
                }
            )

        # Evaluate text metrics (BLEU / ROUGE-L / PPL / eval_time / GPU_mem)
        metrics = evaluate_model_text(
            model,
            test_loader,
            generate_fn=greedy_generate_transformer,
            max_batches=max_eval_batches,
            name=name,
        )

        result_entry = {
            "name": name,
            "num_layers": n_layers,
            "nheads": n_heads,
            "epochs": epochs,
            "total_train_time_sec": float(total_train_time),
            "final_train_loss": float(history[-1]["train_loss"]),
            "final_valid_loss": float(history[-1]["valid_loss"]),
            "metrics": metrics,
        }
        results.append(result_entry)

        # save history for this config
        hist_path = os.path.join(DIRS["logs"], f"ablation_history_{name}.json")
        with open(hist_path, "w", encoding="utf-8") as f:
            json.dump(history, f, indent=2)
        print(f"[{name}] Saved ablation history ->", hist_path)

        # save final model checkpoint (optional)
        ckpt_path = os.path.join(DIRS["checkpoints"], f"transformer_{name}.pt")
        torch.save(model.state_dict(), ckpt_path)
        print(f"[{name}] Saved checkpoint ->", ckpt_path)

    # Save combined results
    ablation_path = os.path.join(DIRS["logs"], "transformer_ablation_results.json")
    with open(ablation_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)
    print("Saved ablation results ->", ablation_path)

    return results


In [35]:
ablation_results = run_transformer_ablation(ABLATION_CONFIGS, epochs=ABLATION_EPOCHS)
ablation_results


[Ablation] Config L2_H2: layers=2, heads=2
[L2_H2] Epoch 1/5 | Train=3.705 | Valid=2.786 | Time=104.6s
[L2_H2] Epoch 2/5 | Train=2.592 | Valid=2.379 | Time=104.4s
[L2_H2] Epoch 3/5 | Train=2.179 | Valid=2.202 | Time=103.2s
[L2_H2] Epoch 4/5 | Train=1.942 | Valid=2.115 | Time=102.1s
[L2_H2] Epoch 5/5 | Train=1.787 | Valid=2.063 | Time=103.0s
[L2_H2] Saved ablation history -> /content/drive/MyDrive/TakeHome543/Question3/logs/ablation_history_L2_H2.json
[L2_H2] Saved checkpoint -> /content/drive/MyDrive/TakeHome543/Question3/checkpoints/transformer_L2_H2.pt
[Ablation] Config L2_H4: layers=2, heads=4
[L2_H4] Epoch 1/5 | Train=3.672 | Valid=2.766 | Time=103.6s
[L2_H4] Epoch 2/5 | Train=2.578 | Valid=2.377 | Time=102.6s
[L2_H4] Epoch 3/5 | Train=2.167 | Valid=2.200 | Time=104.2s
[L2_H4] Epoch 4/5 | Train=1.933 | Valid=2.103 | Time=103.2s
[L2_H4] Epoch 5/5 | Train=1.777 | Valid=2.062 | Time=102.4s
[L2_H4] Saved ablation history -> /content/drive/MyDrive/TakeHome543/Question3/logs/ablation_his

[{'name': 'L2_H2',
  'num_layers': 2,
  'nheads': 2,
  'epochs': 5,
  'total_train_time_sec': 517.1938333511353,
  'final_train_loss': 1.786985621255566,
  'final_valid_loss': 2.062884291013082,
  'metrics': {'name': 'L2_H2',
   'bleu': 23.814198328255337,
   'rougeL': 0.5186164804337138,
   'ppl': 7.969892029884116,
   'eval_time_sec': 9.913009881973267,
   'max_gpu_mem_mb': 1133.0966796875,
   'avg_loss': 2.0756709456443785,
   'num_batches': 30}},
 {'name': 'L2_H4',
  'num_layers': 2,
  'nheads': 4,
  'epochs': 5,
  'total_train_time_sec': 515.970596075058,
  'final_train_loss': 1.7771086102513256,
  'final_valid_loss': 2.0623613524640727,
  'metrics': {'name': 'L2_H4',
   'bleu': 23.861889136925356,
   'rougeL': 0.5224690759966713,
   'ppl': 7.981119074163615,
   'eval_time_sec': 9.062343120574951,
   'max_gpu_mem_mb': 1133.0966796875,
   'avg_loss': 2.077078636487325,
   'num_batches': 30}},
 {'name': 'L4_H4',
  'num_layers': 4,
  'nheads': 4,
  'epochs': 5,
  'total_train_time_se

In [36]:
def plot_ablation_results(results):
    # sort by layers then heads for consistent plotting
    results_sorted = sorted(results, key=lambda x: (x["num_layers"], x["nheads"]))

    layers = [r["num_layers"] for r in results_sorted]
    heads = [r["nheads"] for r in results_sorted]
    bleus = [r["metrics"]["bleu"] for r in results_sorted]
    train_times = [r["total_train_time_sec"] for r in results_sorted]

    # BLEU vs config (x-axis = index with labels)
    labels = [f"L{l}_H{h}" for l, h in zip(layers, heads)]

    plt.figure()
    plt.plot(labels, bleus, marker="o")
    plt.xlabel("Config (Layers_Heads)")
    plt.ylabel("BLEU")
    plt.title("Transformer Ablation: BLEU vs (Layers, Heads)")
    plt.grid(True)
    fig_path1 = os.path.join(DIRS["figs"], "ablation_bleu_vs_config.png")
    plt.savefig(fig_path1, dpi=200, bbox_inches="tight")
    plt.close()
    print("Saved BLEU ablation figure ->", fig_path1)

    # Train time vs config
    plt.figure()
    plt.plot(labels, train_times, marker="o")
    plt.xlabel("Config (Layers_Heads)")
    plt.ylabel("Total train time (s)")
    plt.title("Transformer Ablation: Train Time vs (Layers, Heads)")
    plt.grid(True)
    fig_path2 = os.path.join(DIRS["figs"], "ablation_time_vs_config.png")
    plt.savefig(fig_path2, dpi=200, bbox_inches="tight")
    plt.close()
    print("Saved Train Time ablation figure ->", fig_path2)


plot_ablation_results(ablation_results)


Saved BLEU ablation figure -> /content/drive/MyDrive/TakeHome543/Question3/figs/ablation_bleu_vs_config.png
Saved Train Time ablation figure -> /content/drive/MyDrive/TakeHome543/Question3/figs/ablation_time_vs_config.png
